In [1]:
import torch
import torch.nn as nn
from model import RegressionModel
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from numpy import load
import matplotlib.pyplot as plt

In [32]:
def train_model(organ):

    #data loading and preprocessing
    data = load('training_arrays/abdomen_merged_training_arrays/'+ organ + '_training_array.npy')

    X = data[:,:-1]
    y = data[:,-1]

    # Find rows containing NaN values in X
    nan_rows_X = np.isnan(X).any(axis=1)

    # Drop rows with NaN values from X and y
    X = X[~nan_rows_X]
    y = y[~nan_rows_X]

    # Scaling and splitting data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

    batch_size = 64
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Setting device to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #Setting up model
    hidden_sizes = [512, 256, 128, 64]
    model = RegressionModel(input_size=5, hidden_sizes=hidden_sizes, output_size=1, num_layers=5, dropout=0.1)
    model.to(device) 

    #loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    #training
    num_epochs = 2000

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        model.eval()
        with torch.no_grad():
            val_loss = 0
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets.unsqueeze(1))
                val_loss += loss.item()
            val_loss /= len(val_loader)

        
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    #save model
    torch.save(model.state_dict(), 'abdomen_models/' + organ + '_model.pth')
    


In [ ]:
#Train models for abdomen organs/tissues

organs_list = ['colon', 'kidney_left', 'kidney_right', 'liver', 'pancreas',
                'skin', 'small_bowel', 'spleen', 'stomach', 'urinary_bladder', 
                'adrenal_gland_left', 'adrenal_gland_right', 'spinal_cord']

for organ in organs_list:
    train_model(organ)